In [4]:
import glob
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision.models import squeezenet1_1, resnet50, densenet201
from torch.utils.data import TensorDataset, DataLoader
from preprocessing_pipeline import get_data

In [9]:
num_classes = 3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
train_data, test_data = get_data(num_classes=num_classes)

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

100%|██████████| 12000/12000 [00:11<00:00, 1083.24it/s]


In [30]:
squeezenet_model = squeezenet1_1(weights='DEFAULT')
squeezenet_model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
squeezenet_model.num_classes = num_classes
squeezenet_model = squeezenet_model.to(device)

print(next(squeezenet_model.parameters()).device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=squeezenet_model.parameters(), lr = 1e-5)

training_loss = []
num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    for i, (b_x, b_y) in enumerate(train_dataloader):
        # load data to gpu
        b_x = b_x.to(device)
        b_y = b_y.to(device)
        
        y_pred = squeezenet_model(b_x)
        loss = loss_fn(y_pred, b_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # progress_bar.update(1)
        
        if i % 100 == 0:
            training_loss.append(loss.item())
            print(f"Batch: {i} / Epoch: {epoch} / Loss: {loss.item():.4f} / Pred:{y_pred[0]}")

cuda:0


  0%|          | 0/1500 [06:02<?, ?it/s]

Epoch 1/20
Batch: 0 / Epoch: 0 / Loss: 1.9132 / Pred:tensor([3.1362, 3.9493, 5.1024], device='cuda:0', grad_fn=<SelectBackward0>)


Batch: 100 / Epoch: 0 / Loss: 0.9641 / Pred:tensor([1.1034, 3.3354, 1.9352], device='cuda:0', grad_fn=<SelectBackward0>)
Batch: 200 / Epoch: 0 / Loss: 0.7433 / Pred:tensor([2.6188, 0.2852, 2.0602], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch 2/20
Batch: 0 / Epoch: 1 / Loss: 0.5441 / Pred:tensor([7.8896, 0.0409, 3.3976], device='cuda:0', grad_fn=<SelectBackward0>)
Batch: 100 / Epoch: 1 / Loss: 0.5054 / Pred:tensor([5.7580, 0.7011, 0.9782], device='cuda:0', grad_fn=<SelectBackward0>)
Batch: 200 / Epoch: 1 / Loss: 0.5552 / Pred:tensor([0.2149, 3.6262, 0.5827], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch 3/20
Batch: 0 / Epoch: 2 / Loss: 0.4669 / Pred:tensor([0.2593, 6.8310, 1.0293], device='cuda:0', grad_fn=<SelectBackward0>)
Batch: 100 / Epoch: 2 / Loss: 0.3665 / Pred:tensor([7.0009, 0.1938, 1.4598], device='cuda:0', grad_fn=<SelectBackward0>)
Batch: 200 / Epoch: 2 / Loss: 0.3184 / Pred:tensor([7.3259, 0.0000, 1.8489], device='cuda:0', grad_fn=<SelectBackward0>)
Epoch 4/20
Bat

In [32]:
def validate(model, data):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(data):
        images = images.to(device)
        labels = labels.to(device)
        
        x = model(images)
        value, pred = torch.max(x, 1)
        
        total += x.size(0)
        correct += torch.sum(pred == labels)
        
        if i % 100 == 0:
            print(f"Pred: {x} / True: {labels}")
    return correct / total

In [34]:
validate(squeezenet_model, test_dataloader)

Pred: tensor([[11.5583,  1.3501,  3.8475],
        [10.9674,  0.0000,  7.3296],
        [ 2.7736,  0.0000, 10.9558],
        [ 2.4385,  2.9353,  9.2993],
        [11.4846,  0.2722,  3.9848],
        [ 3.0937,  0.7231, 20.3247],
        [ 4.3358,  5.3869, 10.7116],
        [ 2.0977,  1.1553, 11.8016],
        [ 1.3471,  1.3132, 16.0823],
        [ 4.4009,  3.7499, 21.0649],
        [ 4.5328, 12.7887,  0.4928],
        [ 8.3177,  2.5135,  4.8718],
        [ 1.6772,  0.8711, 14.2273],
        [ 4.8196, 14.5257,  4.3074],
        [ 3.3409,  1.1962, 19.3875],
        [ 7.5941, 18.9709,  2.1439],
        [ 2.6958, 12.3721,  0.9817],
        [ 3.0802,  0.7648, 15.0735],
        [ 2.5445,  2.8574, 12.0561],
        [18.6187,  0.0000,  3.3410],
        [17.0872,  0.0000,  4.3105],
        [ 5.9323, 16.5959,  1.1664],
        [13.3630,  0.1723,  4.1763],
        [ 4.1997, 11.1540,  2.9674],
        [18.3207,  0.0000,  4.9398],
        [17.9952,  0.6253,  1.0803],
        [ 1.7773,  2.5519, 18.41

tensor(0.9775, device='cuda:0')